In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 


In [2]:
dfinfluence = pd.read_csv('input/influence_data.csv', encoding='unicode_escape')

In [3]:
# print(dfinfluence)

## 建立artists,记录全部艺术家

In [4]:
artists = pd.DataFrame(columns= ['name', 'id'])
name2art = {}
id2art = {}
# for index, row in df.iterrows():
for index, row in dfinfluence.iterrows():
    id = row['influencer_id']
    name = row['influencer_name']
    # if (id not in id2art) & (name in name2art):
    #     print('name:', name)
    #     # print(name2art)
    #     artist = artists.loc[name2art[name]]
    #     print(id, ' ', artist['id'])
    # if (id in id2art) & (name not in name2art):
    if id in id2art:
        # print('id:', id)
        artist = artists.loc[id2art[id]]
        if artist['name'] != name:
            print("index=", index, "id=", id, ", names= ", artist['name'], ' ', name)
        else:
            continue
    else:
        artists.loc[artists.shape[0]] = [name, id]
    # artists.append([name, id], ignore_index=True)
    # print(artists)
    # name2art[name] = artists.shape[0] - 1
        id2art[id] = artists.shape[0] - 1

for index, row in dfinfluence.iterrows():
    id = row['follower_id']
    name = row['follower_name']
    if id in id2art:
        artist = artists.loc[id2art[id]]
        if artist['name'] != name:
            print("index=", index, "id= ", id, "names= ", artist['name'], ' ', name)
        else:
            continue
    else:
        artists.loc[artists.shape[0]] = [name, id]
        id2art[id] = artists.shape[0] - 1


index= 6644 id= 893383 , names=  The Muffs   The Kills
index= 29381 id= 303506 , names=  Usher   Day26
index= 37697 id= 957340 , names=  Helloween   Nightwish


## 修改有错误的数据

In [6]:
dfinfluence_ = dfinfluence.copy()
# print(dfinfluence_.loc[6644])
dfinfluence_.at[6644, 'influencer_name'] = 'The Muffs'
# print(dfinfluence_.loc[29381])
dfinfluence_.at[29381, 'influencer_name'] = 'Usher'
# print(dfinfluence_.loc[37697])
dfinfluence_.at[37697, 'influencer_name'] = 'Helloween'
print(dfinfluence_.loc[37697])
print(dfinfluence.loc[37687])

influencer_id                 957340
influencer_name            Helloween
influencer_main_genre       Pop/Rock
influencer_active_start         1980
follower_id                   868628
follower_name              Nightwish
follower_main_genre         Pop/Rock
follower_active_start           1990
Name: 37697, dtype: object
influencer_id                     771438
influencer_name            Black Sabbath
influencer_main_genre           Pop/Rock
influencer_active_start             1960
follower_id                       868616
follower_name                 Nickelback
follower_main_genre             Pop/Rock
follower_active_start               1990
Name: 37687, dtype: object


## 看看一共有多少个艺术家（主要的核查点）

In [5]:
# print(artists)
print(artists.shape)
print(len(id2art))

## 核查可以从下面这些开始
arr1 = dfinfluence['influencer_id'].unique()
# print(len(arr1))
# print(arr2)
# print(arr1 + arr2)
arr2 = dfinfluence['follower_id'].unique()
# list(set(a) | set(b))
arr = list(set(arr1) | set(arr2))
print(len(arr))
arr1 = dfinfluence['influencer_name'].unique()
arr2 = dfinfluence['follower_name'].unique()
arr = list(set(arr1) | set(arr2))
print(len(arr))


(5603, 2)
5603
5603
5568


## 补充artist

In [7]:
# df = pd.DataFrame({'A': [12, 23], 'B': [['a', 'b'], ['c', 'd']]})
# print(df)


artists 中的`towards`和`from`中的值的类型$\in$`artists.index`

In [33]:
# print(id2art)
artists['towards'] = pd.Series([[]] * artists.shape[0], index = artists.index)
artists['from'] = pd.Series([[]] * artists.shape[0], index = artists.index)
# print(pd.Series([[]] * artists.shape[0]))
# print(artists.head())
# artists.at[1, 'from'] = [1]
# print(artists.head())

for index, row in dfinfluence_.iterrows():
    iart_inf = id2art[row['influencer_id']]
    iart_fol = id2art[row['follower_id']]
    artists.at[iart_inf, 'towards'] = artists.at[iart_inf, 'towards'] + [iart_fol]
    artists.at[iart_fol, 'from'] = artists.at[iart_fol, 'from'] + [iart_inf]

print(artists.head())

            name      id                                            towards  \
0  The Exploited  759491  [3774, 3919, 1382, 1383, 1544, 211, 4530, 1364...   
1         Tricky   25462  [489, 1626, 3130, 3545, 4169, 2966, 4367, 1539...   
2      Bob Dylan   66915  [489, 678, 42, 2692, 2816, 1888, 648, 276, 462...   
3  Leonard Cohen   71209  [489, 3223, 840, 1889, 3949, 3682, 2657, 6, 24...   
4   The Gun Club   91438  [489, 947, 3177, 714, 1831, 2954, 17, 1750, 5161]   

                                                from  influenced_numbers  
0  [680, 1171, 1359, 1315, 1838, 1599, 361, 1675,...                   2  
1  [777, 1005, 1006, 478, 1007, 1008, 228, 195, 1...                   0  
2  [1697, 1375, 1446, 1698, 1699, 233, 1700, 1447...                   1  
3  [2, 1486, 1757, 424, 1134, 1054, 480, 425, 673...                   0  
4  [679, 352, 152, 155, 1839, 868, 1568, 852, 194...                   0  


In [34]:
# print(artists.iloc[489])

`influenced_numbers` 为一个artist的影响人数

In [47]:
artists['influenced_numbers'] = pd.Series([0] * artists.shape[0])
for index, artist in artists.iterrows():
    # 被多少人影响
    arr = artist['from']
    influencer_num = len(arr)
    # print(influencer_num)
    if influencer_num == 0:
        continue 
    for influencer in arr:
        # print(art_inf)
        # print(arr)
        artists.loc[influencer, 'influenced_numbers'] += 1 / influencer_num
        # print('now:', artists.at[influencer, 'influenced_numbers'])

print(artists.head(20))

                         name      id  \
0               The Exploited  759491   
1                      Tricky   25462   
2                   Bob Dylan   66915   
3               Leonard Cohen   71209   
4                The Gun Club   91438   
5              Chrissie Hynde  113823   
6   Crime & the City Solution  132710   
7                 Lesley Gore  208582   
8              Massive Attack  378288   
9                   Nick Cave  397880   
10              The Raincoats  489999   
11                 Pretenders  492331   
12                  Tom Waits  615119   
13         Marianne Faithfull  651107   
14                Patti Smith  747445   
15                Sonic Youth  755156   
16                   Pere Ubu  838272   
17                     Pixies  895136   
18                 The Smiths  899530   
19                          X  960690   

                                              towards  \
0   [3774, 3919, 1382, 1383, 1544, 211, 4530, 1364...   
1   [489, 1626, 3130, 35